<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto City</font></h1>

In [3]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np
!conda install -c conda-forge folium=0.5.0 --yes
import folium
import json
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==5.3.1=py37_0
  - defaults/linux-64::astropy==3.0.4=py37h14c3975_0
  - defaults/linux-64::bkcharts==0.2=py37_0
  - defaults/linux-64::blaze==0.11.3=py37_0
  - defaults/linux-64::bokeh==0.13.0=py37_0
  - defaults/linux-64::bottleneck==1.2.1=py37h035aef0_1
  - defaults/linux-64::dask==0.19.1=py37_0
  - defaults/linux-64::datashape==0.5.4=py37_1
  - defaults/linux-64::mkl-service==1.1.2=py37h90e4bf4_5
  - defaults/linux-64::numba==0.39.0=py37h04863e7_0
  - defaults/linux-64::numexpr==2.6.8=py37hd89afb7_0
  - defaults/linux-64::odo==0.5.1=py37_0
  - defaults/linux-64::pytables==3.4.4=py37ha205bf6_0
  - defaults/linux-64::pytest-arraydiff==0.2=py37h39e3cac_0
  - defaults/linux-64::pytest-astropy==0.4.0=py37_0
  - defaults/linux-64::pytest-doctestplus==0.1.3=py37_0
  - defaults/linux-64::pywavelets==1.0.0=py37h

# 1. Web scraping Data and creating a Dataframe

In [4]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = bs(source, 'lxml')

In [5]:
table = soup.find('table')

In [6]:
newdata=[]
tbody=soup.find('tbody')
for text in tbody.stripped_strings:
    newdata.append(text)
newdata[0:6]

['Postcode', 'Borough', 'Neighbourhood', 'M1A', 'Not assigned', 'Not assigned']

In [7]:
del newdata[0:3]
newdata[0:6]

['M1A', 'Not assigned', 'Not assigned', 'M2A', 'Not assigned', 'Not assigned']

In [8]:
len(newdata)

864

In [9]:
postcode= newdata[0:len(newdata):3]
borough= newdata[1:len(newdata):3]
neighbourhood= newdata[2:len(newdata):3]

In [10]:
df = pd.DataFrame({'PostCode':postcode,'Borough':borough,'Neighbourhood':neighbourhood})

In [11]:
df.head()

,PostCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [12]:
df.replace({'Borough':'Not assigned'},np.nan,inplace=True)
df.head()

,PostCode,Borough,Neighbourhood
0,M1A,NaN,Not assigned
1,M2A,NaN,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [13]:
df.dropna(how='any',inplace=True)
df.reset_index(inplace=True)
df.head()

,index,PostCode,Borough,Neighbourhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,Harbourfront
3,5,M5A,Downtown Toronto,Regent Park
4,6,M6A,North York,Lawrence Heights


In [14]:
df.drop('index', axis=1,inplace=True)
df.head()

,PostCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [15]:
df.loc[df.Neighbourhood=='Not assigned', :]

,PostCode,Borough,Neighbourhood
6,M7A,Queen's Park,Not assigned


In [16]:
df['Neighbourhood'].replace(['Not assigned'],["Queen's Park"], inplace=True)
df.head(8)

,PostCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue


In [17]:
len(df['PostCode'].unique())

103

In [18]:
df= df.groupby(['PostCode','Borough'])[['Neighbourhood']].sum()
df.reset_index(inplace=True)

In [19]:
df.head()

,PostCode,Borough,Neighbourhood
0,M1B,Scarborough,RougeMalvern
1,M1C,Scarborough,Highland CreekRouge HillPort Union
2,M1E,Scarborough,GuildwoodMorningsideWest Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# 2.Merging the Data frame with location data

In [20]:
df.shape

(103, 3)

In [21]:
df2 = pd.read_csv('http://cocl.us/Geospatial_data')
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [24]:
df3= pd.merge(df,df2,on='PostCode')
df3.head()

,PostCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,RougeMalvern,43.806686,-79.194353
1,M1C,Scarborough,Highland CreekRouge HillPort Union,43.784535,-79.160497
2,M1E,Scarborough,GuildwoodMorningsideWest Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [23]:
df2.rename(columns={'Postal Code':'PostCode'}, inplace=True)
df2.head()

,PostCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [25]:
latitude=43.5
longitude=-79.30
map_can = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df3['Latitude'], df3['Longitude'], df3['Borough'], df3['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='yellow',
        fill_opacity=0.7,
        parse_html=False).add_to(map_can)  
    
map_can

# 3.Create a map of Toronto with neighborhoods superimposed on top.

In [26]:
toronto_data = df3[(df3.Borough == 'West Toronto') | (df3.Borough == 'York') | (df3.Borough == 'North York')].reset_index(drop=True)
toronto_data.head()

,PostCode,Borough,Neighbourhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,North York,FairviewHenry FarmOriole,43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,North York,Silver HillsYork Mills,43.757490,-79.374714
4,M2M,North York,NewtonbrookWillowdale,43.789053,-79.408493


# 4. Slicing the dataframe to create a new dataframe of the 3 boroughs - _West toronto, York & North York_.

In [27]:
latitude=43.7
longitude=-79.30
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [29]:
CLIENT_ID = 'JOJGDNM2CCM0ZEIXTZ4GW210BTB4L2E3PPTZK5GYLTTOI0PW' # your Foursquare ID
CLIENT_SECRET = 'Q2TX2S0ETV3JVY3Y1OS0XUBVUMC03YFATP0W250KKML2N3JJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version



In [30]:
neighbourhood_latitude = toronto_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighbourhood_longitude = toronto_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighbourhood_name = toronto_data.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Hillcrest Village are 43.8037622, -79.3634517.


# 5. Explore Neighborhoods in Toronto

In [31]:
# type your answer here
limit = 100
radius=500
url= 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, neighbourhood_latitude, neighbourhood_longitude, radius, limit)
url




'https://api.foursquare.com/v2/venues/explore?&client_id=JOJGDNM2CCM0ZEIXTZ4GW210BTB4L2E3PPTZK5GYLTTOI0PW&client_secret=Q2TX2S0ETV3JVY3Y1OS0XUBVUMC03YFATP0W250KKML2N3JJ&v=20180605&ll=43.8037622,-79.3634517&radius=500&limit=100'

In [32]:
results = requests.get(url).json()

In [33]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [34]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Eagle's Nest Golf Club,Golf Course,43.805455,-79.364186
1,AY Jackson Pool,Pool,43.804515,-79.366138
2,Villa Madina,Mediterranean Restaurant,43.801685,-79.363938
3,Duncan Creek Park,Dog Run,43.805539,-79.360695


In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [36]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Hillcrest Village
FairviewHenry FarmOriole
Bayview Village
Silver HillsYork Mills
NewtonbrookWillowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon ParkDon Mills South
Bathurst ManorDownsview NorthWilson Heights
Northwood ParkYork University
CFB TorontoDownsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Bedford ParkLawrence Manor East
Lawrence HeightsLawrence Manor
Glencairn
Humewood-Cedarvale
Caledonia-Fairbanks
Dovercourt VillageDufferin
Little PortugalTrinity
BrocktonExhibition PlaceParkdale Village
DownsviewNorth ParkUpwood Park
Del RayKeelesdaleMount DennisSilverthorn
The Junction NorthRunnymede
High ParkThe Junction South
ParkdaleRoncesvalles
RunnymedeSwansea
Humber Summit
EmeryHumberlea
Weston


In [37]:
toronto_venues.head()


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
2,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
3,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run
4,FairviewHenry FarmOriole,43.778517,-79.346556,The LEGO Store,43.778207,-79.343483,Toy / Game Store


In [38]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Airport,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,...,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,FairviewHenry FarmOriole,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


# 6. Analyzing Each Neighborhood

In [39]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Airport,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,...,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Bathurst ManorDownsview NorthWilson Heights,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.00,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000
1,Bayview Village,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Bedford ParkLawrence Manor East,0.0000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,BrocktonExhibition PlaceParkdale Village,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.045455,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455
4,CFB TorontoDownsview East,0.0000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Caledonia-Fairbanks,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000
6,Del RayKeelesdaleMount DennisSilverthorn,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Don Mills North,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Dovercourt VillageDufferin,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.100000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Downsview Central,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [40]:
toronto_grouped.shape

(34, 155)

In [41]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst ManorDownsview NorthWilson Heights----
                       venue  freq
0                Coffee Shop  0.11
1              Deli / Bodega  0.06
2  Middle Eastern Restaurant  0.06
3        Fried Chicken Joint  0.06
4         Frozen Yogurt Shop  0.06


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1   Chinese Restaurant  0.25
2                 Bank  0.25
3                 Café  0.25
4          Music Venue  0.00


----Bedford ParkLawrence Manor East----
                  venue  freq
0    Italian Restaurant  0.08
1           Coffee Shop  0.08
2  Fast Food Restaurant  0.08
3                   Pub  0.04
4           Pizza Place  0.04


----BrocktonExhibition PlaceParkdale Village----
            venue  freq
0  Breakfast Spot  0.09
1     Coffee Shop  0.09
2            Café  0.09
3             Gym  0.05
4         Stadium  0.05


----CFB TorontoDownsview East----
                venue  freq
0             Airport  0.33
1                Park  0.33
2

## Printing each neighborhood along with the top 5 most common venues

In [42]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## Puting this data into a dataframe.

In [80]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Bathurst ManorDownsview NorthWilson Heights,Coffee Shop,Pharmacy,Bank,Diner,Sandwich Place
1,Bayview Village,Bank,Café,Chinese Restaurant,Japanese Restaurant,Empanada Restaurant
2,Bedford ParkLawrence Manor East,Coffee Shop,Italian Restaurant,Fast Food Restaurant,Grocery Store,Indian Restaurant
3,BrocktonExhibition PlaceParkdale Village,Coffee Shop,Breakfast Spot,Café,Convenience Store,Pet Store
4,CFB TorontoDownsview East,Airport,Park,Other Repair Shop,Yoga Studio,Dim Sum Restaurant


In [81]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 3, 3, 1, 1, 1, 1], dtype=int32)

# 7.Clustering Neighbourhoods

In [82]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'ClusterLabels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,PostCode,Borough,Neighbourhood,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,1.0,Golf Course,Pool,Dog Run,Mediterranean Restaurant,Yoga Studio
1,M2J,North York,FairviewHenry FarmOriole,43.778517,-79.346556,1.0,Clothing Store,Fast Food Restaurant,Coffee Shop,Restaurant,Tea Room
2,M2K,North York,Bayview Village,43.786947,-79.385975,1.0,Bank,Café,Chinese Restaurant,Japanese Restaurant,Empanada Restaurant
3,M2L,North York,Silver HillsYork Mills,43.757490,-79.374714,4.0,Cafeteria,Yoga Studio,Empanada Restaurant,Electronics Store,Eastern European Restaurant
4,M2M,North York,NewtonbrookWillowdale,43.789053,-79.408493,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
toronto_merged.drop(4,inplace=True)

In [87]:
toronto_merged.reset_index()
toronto_merged.head(6)

,PostCode,Borough,Neighbourhood,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,1.0,Golf Course,Pool,Dog Run,Mediterranean Restaurant,Yoga Studio
1,M2J,North York,FairviewHenry FarmOriole,43.778517,-79.346556,1.0,Clothing Store,Fast Food Restaurant,Coffee Shop,Restaurant,Tea Room
2,M2K,North York,Bayview Village,43.786947,-79.385975,1.0,Bank,Café,Chinese Restaurant,Japanese Restaurant,Empanada Restaurant
3,M2L,North York,Silver HillsYork Mills,43.757490,-79.374714,4.0,Cafeteria,Yoga Studio,Empanada Restaurant,Electronics Store,Eastern European Restaurant
5,M2N,North York,Willowdale South,43.770120,-79.408493,1.0,Coffee Shop,Sushi Restaurant,Ramen Restaurant,Restaurant,Café
6,M2P,North York,York Mills West,43.752758,-79.400049,3.0,Park,Bank,Bar,Convenience Store,Empanada Restaurant


In [89]:
toronto_merged['ClusterLabels'] = toronto_merged.ClusterLabels.astype(int)

In [96]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['ClusterLabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color='black',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Creating a map with clustered data

In [99]:
toronto_merged.loc[toronto_merged['ClusterLabels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
33,North York,2,Baseball Field,Yoga Studio,Fast Food Restaurant,Event Space,Empanada Restaurant


# 8. Examining Clusters

In [97]:
toronto_merged.loc[toronto_merged['ClusterLabels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
14,North York,0,Grocery Store,Shopping Mall,Bank,Convenience Store,Diner


In [98]:
toronto_merged.loc[toronto_merged['ClusterLabels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,North York,1,Golf Course,Pool,Dog Run,Mediterranean Restaurant,Yoga Studio
1,North York,1,Clothing Store,Fast Food Restaurant,Coffee Shop,Restaurant,Tea Room
2,North York,1,Bank,Café,Chinese Restaurant,Japanese Restaurant,Empanada Restaurant
5,North York,1,Coffee Shop,Sushi Restaurant,Ramen Restaurant,Restaurant,Café
7,North York,1,Grocery Store,Pharmacy,Discount Store,Coffee Shop,Pizza Place
9,North York,1,Baseball Field,Gym / Fitness Center,Caribbean Restaurant,Japanese Restaurant,Café
10,North York,1,Gym,Beer Store,Asian Restaurant,Coffee Shop,Grocery Store
11,North York,1,Coffee Shop,Pharmacy,Bank,Diner,Sandwich Place
12,North York,1,Coffee Shop,Falafel Restaurant,Massage Studio,Caribbean Restaurant,Miscellaneous Shop
15,North York,1,Korean Restaurant,Home Service,Food Truck,Baseball Field,Yoga Studio


In [100]:
toronto_merged.loc[toronto_merged['ClusterLabels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
6,North York,3,Park,Bank,Bar,Convenience Store,Empanada Restaurant
8,North York,3,Fast Food Restaurant,Park,Food & Drink Shop,Yoga Studio,Dim Sum Restaurant
13,North York,3,Airport,Park,Other Repair Shop,Yoga Studio,Dim Sum Restaurant
20,North York,3,Pizza Place,Park,Pub,Japanese Restaurant,Department Store
22,York,3,Park,Women's Store,Pharmacy,Fast Food Restaurant,Market
26,North York,3,Construction & Landscaping,Park,Deli / Bodega,Bakery,Basketball Court
34,York,3,Park,Yoga Studio,Dessert Shop,Empanada Restaurant,Electronics Store


In [101]:
toronto_merged.loc[toronto_merged['ClusterLabels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,North York,4,Cafeteria,Yoga Studio,Empanada Restaurant,Electronics Store,Eastern European Restaurant
